<a href="https://colab.research.google.com/github/rrwiren/ilmanlaatu-ennuste-helsinki/blob/main/SARIMAX_pilkottu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 0. Esitiedot ja Tavoite (SARIMAX)
"""
Otsoniennuste Helsinki - SARIMAX-malli

Tavoite:
1. Ladata uusi esikäsitelty data (sis. pilvisyys) Parquet-tiedostosta.
2. Sovittaa SARIMAX (Seasonal AutoRegressive Integrated Moving Average
   with eXogenous regressors) -malli Otsoni [µg/m³] -aikasarjaan.
3. Mallissa hyödynnetään sekä otsonin omaa historiaa, kausivaihtelua (S)
   että ulkoisia selittäviä muuttujia (X), kuten säädataa.
4. Tehdä ennusteita testijaksolle (seuraavat 24 tuntia).
5. Arvioida mallin suorituskykyä (RMSE, MAE) ja verrata baselineen sekä
   aiempaan ARIMA-malliin. Tarkastellaan myös 8h varoitusrajan ennustamista.

HUOM: Tulevaisuuden ennusteiden tekeminen SARIMAXilla vaatii myös
ennusteet ulkoisille muuttujille (sääennusteet) ennustejaksolle.
Tässä versiossa käytämme yksinkertaistettua lähestymistapaa tähän.
"""
print("Osa 0: Esitiedot (SARIMAX) - OK")

In [ ]:
# @title 1. Tuonnit ja Asetukset (SARIMAX)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
import os
import warnings
import traceback

# Tilastollinen mallinnus ja aikasarja-analyysi
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# *** MUUTOS: Tuodaan SARIMAX ARIMAn sijaan/lisäksi ***
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Arviointimetriikat
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report

# Asetukset kuvaajille
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Estetään turhia varoituksia
warnings.filterwarnings("ignore")

# --- Data-asetukset ---
BASE_GITHUB_URL = 'https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/'
# Varmistetaan oikea tiedostonimi (v3 sisälsi pilvisyyden)
PARQUET_PATH = 'data/processed/processed_Helsinki_O3_Weather_Cloudiness_2024_2025_v3.parquet'
DATA_URL = BASE_GITHUB_URL + PARQUET_PATH

TARGET_COLUMN = 'Otsoni [µg/m³]' # Ennustettava sarake

# *** UUSI: Määritellään ulkoiset regressorimuuttujat (säädata yms.) ***
# Otetaan mukaan kaikki muut sarakkeet paitsi itse kohdemuuttuja
# Huom: Feature engineeringiä (esim. sin/cos muunnoksia ajalle/tuulelle)
# kannattaa harkita jo esikäsittelyssä TAI tässä ennen mallinnusta.
# Oletetaan, että esikäsittely on tehnyt tarvittavat muunnokset, jos niitä halutaan käyttää.
# Jos haluat käyttää feature engineeringiä tässä, lisää vaiheet Osaan 2.
# Tässä oletetaan, että käytämme suoraan Parquet-tiedoston sarakkeita (pl. Otsoni).
EXOG_COLUMNS = [
    'Lämpötilan keskiarvo [°C]',
    'Keskituulen nopeus [m/s]',
    'Ilmanpaineen keskiarvo [hPa]',
    'Tuulen suunnan keskiarvo [°]', # Harkitse sin/cos muunnosta jos ei tehty esikäsittelyssä
    'Pilvisyys [okta]'
    # Lisää tähän muita säämuuttujia tai aikapiirteitä tarvittaessa
    # Esim. jos esikäsittely teki sin/cos ajalle: 'hour_sin', 'hour_cos' jne.
]

# --- Mallinnusasetukset ---
FORECAST_HORIZON = 24
TEST_SPLIT_RATIO = 0.15
# TEST_START_DATE = '2025-03-15' # Vaihtoehtoinen

# SARIMAX-parametrit
# Normaali järjestys: (p, d, q)
ORDER = (1, 0, 1) # Placeholder, d=0 oletus edellisestä ARIMA-ajosta, p/q arvioidaan ACF/PACF
# Kausiluonteinen järjestys: (P, D, Q, m)
# m = kausijakson pituus (24 tuntia vuorokausirytmille)
# P, D, Q määritetään kausiluonteisen differoinnin ja ACF/PACF:n perusteella
SEASONAL_ORDER = (1, 0, 1, 24) # Placeholder (P, D, Q, m) - D=0 oletus, P/Q arvioidaan

# Huom: Koska pmdarima ei ole käytössä, nämä (p,q,P,Q) pitää päätellä
# manuaalisesti ACF/PACF-kuvaajista myöhemmissä vaiheissa.

# --- Varoitusraja (sama kuin ennen) ---
O3_THRESHOLD_8H_AVG = 120 # µg/m³

# --- Satunnaisuuden kiinnitys ---
SEED = 42
np.random.seed(SEED)
# (torch ei tarvita SARIMAXissa)

print("Osa 1: Tuonnit ja Asetukset (SARIMAX) - OK")
print(f"Käytettävät ulkoiset regressorimuuttujat (Exog): {EXOG_COLUMNS}")

In [ ]:
# @title 2. Datan Lataus ja Käsittely (SARIMAX)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
import traceback

print("--- Aloitetaan Datan Lataus ja Käsittely SARIMAXia varten ---")

# Alustetaan muuttujat
df_processed = None
y = None
X = None
y_train, y_test, X_train, X_test = [None]*4 # Käytetään monikon purkua alustukseen

try:
    # 1. Lataa esikäsitelty data Parquet-tiedostosta
    print(f"Ladataan dataa: {DATA_URL}")
    response = requests.get(DATA_URL)
    response.raise_for_status()
    df_processed = pd.read_parquet(io.BytesIO(response.content))
    print(f"Data ladattu, muoto: {df_processed.shape}")

    # 2. Varmista DatetimeIndex ja frekvenssi
    if not isinstance(df_processed.index, pd.DatetimeIndex):
        print("Indeksi ei ole DatetimeIndex. Yritetään muuntaa...")
        df_processed.index = pd.to_datetime(df_processed.index)

    # Yritetään asettaa frekvenssi, jos puuttuu (tärkeää SARIMAXille)
    if df_processed.index.freq is None:
        inferred_freq = pd.infer_freq(df_processed.index)
        print(f"Päätelty datan tiheys: {inferred_freq}")
        if inferred_freq == 'h' or inferred_freq == 'H':
            print("Asetetaan indeksin tiheydeksi 'H'...")
            try:
                df_processed = df_processed.asfreq('H')
                print("Indeksin tiheys asetettu.")
                # Tarkista tuliko NaNneja
                if df_processed.isnull().any().any():
                    print("VAROITUS: asfreq('H') lisäsi NaN-arvoja. Täytetään ffill/bfill...")
                    df_processed.ffill(inplace=True)
                    df_processed.bfill(inplace=True)
                    if df_processed.isnull().any().any():
                         print("VIRHE: Ei voitu täyttää asfreq:n lisäämiä NaN-arvoja.")
                         raise ValueError("NaN-arvoja asfreq:n jälkeen.")
            except Exception as e_freq:
                 print(f"VAROITUS: Indeksin tiheyden asetus epäonnistui: {e_freq}. Mallinnus voi silti toimia.")
        else:
             print("VAROITUS: Datan tiheys ei ole tunti ('H'). SARIMAX voi vaatia tuntidataa.")

    df_processed.sort_index(inplace=True)
    print(f"Data aikaväliltä: {df_processed.index.min()} - {df_processed.index.max()}")

    # 3. Varmista sarakkeiden olemassaolo
    required_cols = [TARGET_COLUMN] + EXOG_COLUMNS
    missing_cols = [col for col in required_cols if col not in df_processed.columns]
    if missing_cols:
        print(f"VIRHE: Seuraavat vaaditut sarakkeet puuttuvat datasta: {missing_cols}")
        raise KeyError("Vaadittuja sarakkeita puuttuu.")

    # 4. Erottele kohdemuuttuja (y) ja ulkoiset regressorimuuttujat (X)
    print(f"Erotellaan kohdemuuttuja '{TARGET_COLUMN}'...")
    y = df_processed[TARGET_COLUMN].copy()

    print(f"Erotellaan ulkoiset regressorimuuttujat (Exog): {EXOG_COLUMNS}...")
    X = df_processed[EXOG_COLUMNS].copy()

    # 5. Käsittele mahdolliset NaN-arvot (varmuuden vuoksi)
    initial_len = len(y)
    y.dropna(inplace=True)
    X = X.loc[y.index] # Pidä X synkronoituna y:n kanssa NaN-poiston jälkeen
    removed_na_y = initial_len - len(y)
    if removed_na_y > 0: print(f"Poistettu {removed_na_y} NaN-riviä kohdemuuttujasta y.")

    initial_len_X = len(X)
    X.dropna(inplace=True)
    y = y.loc[X.index] # Pidä y synkronoituna X:n kanssa
    removed_na_X = initial_len_X - len(X)
    if removed_na_X > 0: print(f"Poistettu {removed_na_X} NaN-riviä regressorimuuttujista X.")

    if y.empty or X.empty:
        raise ValueError("Kohdemuuttuja (y) tai regressorimuuttujat (X) ovat tyhjiä NaN-poiston jälkeen.")

    # Varmistetaan, että indeksit täsmäävät lopullisesti
    common_index = y.index.intersection(X.index)
    y = y.loc[common_index]
    X = X.loc[common_index]
    print(f"Lopullinen datan pituus (y ja X): {len(y)}")

    # 6. Jaa data harjoitus- ja testijoukkoihin (sekä y että X)
    print("\nJaetaan data harjoitus- ja testijoukkoihin...")
    split_index = int(len(y) * (1 - TEST_SPLIT_RATIO))
    y_train = y[:split_index]
    y_test = y[split_index:]
    X_train = X[:split_index]
    X_test = X[split_index:]

    # TAI päivämäärään perustuva jako:
    # if 'TEST_START_DATE' in locals() and TEST_START_DATE is not None:
    #    print(f"Käytetään testijakson aloituspäivää: {TEST_START_DATE}")
    #    try:
    #        test_start_dt = pd.Timestamp(TEST_START_DATE, tz='Europe/Helsinki')
    #        y_train = y[y.index < test_start_dt]
    #        y_test = y[y.index >= test_start_dt]
    #        X_train = X[X.index < test_start_dt]
    #        X_test = X[X.index >= test_start_dt]
    #    except Exception as e_date:
    #        print(f"VIRHE testijakson aloituspäivän käytössä: {e_date}. Käytetään ratio-jakoa.")
    #        # Ratio-jako on jo tehty ylempänä oletuksena
    # else: # Ratio-jako (jo tehty)
    #     pass


    print(f"y_train: {len(y_train)} havaintoa ({y_train.index.min()} - {y_train.index.max()})")
    print(f"y_test:  {len(y_test)} havaintoa ({y_test.index.min()} - {y_test.index.max()})")
    print(f"X_train: {X_train.shape}")
    print(f"X_test:  {X_test.shape}")

    if y_train.empty or X_train.empty or y_test.empty or X_test.empty:
         raise ValueError("Harjoitus- tai testidata (y tai X) on tyhjä jaon jälkeen.")

    if len(y_test) < FORECAST_HORIZON:
        print(f"VAROITUS: Testidata y ({len(y_test)}) on lyhyempi kuin ennustehorisontti ({FORECAST_HORIZON}).")


    # 7. Visualisoi kohdemuuttuja ja yksi regressorimuuttuja
    print("\nVisualisoidaan kohdemuuttuja ja yksi regressorimuuttuja...")
    fig, ax1 = plt.subplots(figsize=(14, 7))

    color = 'tab:blue'
    ax1.set_xlabel('Aika')
    ax1.set_ylabel(TARGET_COLUMN, color=color)
    ax1.plot(y_train.index, y_train, color=color, label='Otsoni (Train)')
    ax1.plot(y_test.index, y_test, color='cyan', label='Otsoni (Test)')
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.legend(loc='upper left')
    ax1.grid(True, linestyle=':')

    # Otetaan esimerkiksi lämpötila toiselle akselille
    exog_to_plot = 'Lämpötilan keskiarvo [°C]'
    if exog_to_plot in X.columns:
        ax2 = ax1.twinx()  # Luo toinen Y-akseli, joka jakaa saman X-akselin
        color = 'tab:red'
        ax2.set_ylabel(exog_to_plot, color=color)
        ax2.plot(X_train.index, X_train[exog_to_plot], color=color, alpha=0.6, linestyle='--', label=f'{exog_to_plot} (Train)')
        ax2.plot(X_test.index, X_test[exog_to_plot], color='magenta', alpha=0.6, linestyle=':', label=f'{exog_to_plot} (Test)')
        ax2.tick_params(axis='y', labelcolor=color)
        ax2.legend(loc='upper right')

    plt.title('Otsoni ja Lämpötila Aikasarjat (Harjoitus/Testi)')
    fig.tight_layout() # Estää otsikoiden menemisen päällekkäin
    plt.show()

    print("\nOsa 2: Datan Lataus ja Käsittely (SARIMAX) - OK")

except KeyError as e:
     print(f"\nVIRHE: Saraketta ei löytynyt: {e}. Tarkista TARGET_COLUMN ja EXOG_COLUMNS.")
     y, X, y_train, y_test, X_train, X_test = [None]*6
except FileNotFoundError:
     print(f"\nVIRHE: Tiedostoa ei löytynyt paikallisesta polusta eikä lataus URL:sta onnistunut.")
     y, X, y_train, y_test, X_train, X_test = [None]*6
except requests.exceptions.RequestException as e:
     print(f"\nVIRHE: Datan lataus URL:sta epäonnistui: {e}")
     y, X, y_train, y_test, X_train, X_test = [None]*6
except ValueError as e:
     print(f"\nVIRHE datan käsittelyssä (ValueError): {e}")
     y, X, y_train, y_test, X_train, X_test = [None]*6
except Exception as e:
     print(f"\nOdottamaton VIRHE datan latauksessa tai käsittelyssä: {e}")
     traceback.print_exc()
     y, X, y_train, y_test, X_train, X_test = [None]*6

In [ ]:
# @title 3. Stationaarisuuden Tarkastelu (y_train) ja Differointi (d) (SARIMAX)

from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np # Varmistetaan numpy tuonti

print("--- Aloitetaan Stationaarisuuden Tarkastelu (y_train) ---")

# Alustetaan muuttujat
d = None
y_train_stationary = None

# Varmistetaan, että y_train on olemassa edellisestä solusta
if 'y_train' in locals() and isinstance(y_train, pd.Series) and not y_train.empty:

    # Funktio ADF-testin suorittamiseen (määritellään uudelleen selkeyden vuoksi)
    def perform_adf_test(series, series_name="Series"):
        """Suorittaa ADF-testin ja tulostaa tulokset selkeästi."""
        print(f"\nSuoritetaan ADF-testi sarjalle: {series_name}")
        # Poista NaN-arvot ennen testiä
        result = adfuller(series.dropna())
        print(f'ADF Statistic: {result[0]:.4f}')
        print(f'p-value: {result[1]:.4f}')
        print('Critical Values:')
        for key, value in result[4].items():
            print(f'\t{key}: {value:.4f}')
        if result[1] <= 0.05:
            print(f"-> Tulos: Hylätään H0 (p <= 0.05). Sarja '{series_name}' on todennäköisesti stationaarinen.")
            return True
        else:
            print(f"-> Tulos: Ei voida hylätä H0 (p > 0.05). Sarja '{series_name}' EI todennäköisesti ole stationaarinen.")
            return False

    # 1. Testaa alkuperäinen y_train
    is_stationary_orig = perform_adf_test(y_train, "Alkuperäinen y_train")
    d = 0 # Oletus
    y_train_stationary = y_train # Oletus

    # 2. Jos ei stationaarinen, kokeile 1. differenssiä
    if not is_stationary_orig:
        print("\nDifferoidaan sarja (1. kertaluku)...")
        y_train_diff1 = y_train.diff().dropna()
        if y_train_diff1.empty:
             print("VIRHE: Differoinnin jälkeen sarja on tyhjä.")
             d = None # Estä jatko
        else:
             is_stationary_diff1 = perform_adf_test(y_train_diff1, "y_train (1. differenssi)")
             if is_stationary_diff1:
                 d = 1
                 y_train_stationary = y_train_diff1
                 print(f"\n-> Ei-kausittaiseksi differoinnin kertaluvuksi (d) asetetaan: {d}")
             else:
                 # 3. Jos vieläkään ei stationaarinen, kokeile 2. differenssiä
                 print("\nDifferoidaan sarja (2. kertaluku)...")
                 y_train_diff2 = y_train_diff1.diff().dropna()
                 if y_train_diff2.empty:
                      print("VIRHE: Toisen differoinnin jälkeen sarja on tyhjä.")
                      d = None # Estä jatko
                 else:
                      is_stationary_diff2 = perform_adf_test(y_train_diff2, "y_train (2. differenssi)")
                      if is_stationary_diff2:
                          d = 2
                          y_train_stationary = y_train_diff2
                          print(f"\n-> Ei-kausittaiseksi differoinnin kertaluvuksi (d) asetetaan: {d}")
                      else:
                          print("\nVAROITUS: Sarja ei näytä tulevan stationaariseksi edes toisella differoinnilla.")
                          d = 2 # Käytetään silti arvoa 2, mutta varoen
                          y_train_stationary = y_train_diff2
                          print(f"\n-> Ei-kausittaiseksi differoinnin kertaluvuksi (d) asetetaan: {d} (varoen)")
    else:
         print(f"\n-> Ei-kausittaiseksi differoinnin kertaluvuksi (d) asetetaan: {d}")


    # 4. Visualisoi (jos differointia tehtiin)
    if d is not None and d > 0:
        plt.figure(figsize=(14, 8))
        plt.subplot(2, 1, 1)
        plt.plot(y_train.index, y_train, label='Alkuperäinen y_train')
        plt.title('Alkuperäinen Otsoni (Harjoitusdata)')
        plt.legend()
        plt.grid(True, linestyle=':')

        plt.subplot(2, 1, 2)
        plt.plot(y_train_stationary.index, y_train_stationary, label=f'y_train ({d}. differenssi)', color='orange')
        plt.title(f'{d}. Asteen Differoitu Aikasarja')
        plt.legend()
        plt.grid(True, linestyle=':')
        plt.tight_layout()
        plt.show()
    elif d == 0:
        print("\nAlkuperäinen sarja on stationaarinen, ei tarvetta differoinnin visualisoinnille.")
        # Varmistetaan että y_train_stationary on asetettu
        if 'y_train_stationary' not in locals() or y_train_stationary is None:
             y_train_stationary = y_train

    # Päivitetään ORDER-muuttuja löydetyllä d:llä
    if d is not None:
         if 'ORDER' in locals() and isinstance(ORDER, tuple) and len(ORDER) == 3:
              ORDER = (ORDER[0], d, ORDER[2]) # Päivitetään vain d-arvo
              print(f"\nPäivitetty ORDER: {ORDER} (p ja q ovat vielä placeholdereita)")
         else:
              print(f"VAROITUS: ORDER-muuttujaa ei löytynyt tai se on vääränlainen. Asetetaan ORDER=(1, {d}, 1).")
              ORDER = (1, d, 1) # Käytä oletus p=1, q=1
    else:
         print("VAROITUS: Differoinnin astetta d ei voitu määrittää.")


    print("\nOsa 3: Stationaarisuuden Tarkastelu (SARIMAX) - OK")

else:
    print("VIRHE: Harjoitusdataa ('y_train') ei löytynyt edellisestä solusta. Aja Osa 2 uudelleen.")
    # Asetetaan d ja data Noneksi, jos data puuttuu
    d = None
    y_train_stationary = None

In [ ]:
# @title 4. Ei-Kausittaisten Kertalukujen (p, q) Määritys (ACF/PACF) (SARIMAX)

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
import traceback

print("--- Määritetään Ei-Kausittaisia ARIMA kertalukuja p ja q (d=0) ---")

# Varmistetaan, että stationaarinen data on olemassa edellisestä solusta
# Koska d=0, käytämme alkuperäistä harjoitusdataa (joka on tallessa y_train_stationary -muuttujassa)
if 'y_train_stationary' in locals() and y_train_stationary is not None and not y_train_stationary.empty:

    # Asetetaan piirrettävien viiveiden (lags) määrä
    # Esim. 72 näyttää 3 päivän korrelaatiot tunneittain
    n_lags = 72

    print(f"\nPiirretään ACF ja PACF {n_lags} viiveellä stationaariselle y_train -datalle...")

    fig, axes = plt.subplots(2, 1, figsize=(12, 8))

    # Autokorrelaatiofunktio (ACF) - auttaa määrittämään q:n
    try:
        # dropna() varmistaa, ettei mukana ole NaN-arvoja (vaikka aiemmin putsattiin)
        plot_acf(y_train_stationary.dropna(), lags=n_lags, ax=axes[0], title=f'Autokorrelaatiofunktio (ACF) (d={d})')
        axes[0].grid(True, linestyle=':')
    except Exception as e_acf:
        print(f"VIRHE ACF-kuvaajan piirrossa: {e_acf}")
        axes[0].set_title('ACF-kuvaajan piirto epäonnistui')
        traceback.print_exc()


    # Osittaisautokorrelaatiofunktio (PACF) - auttaa määrittämään p:n
    try:
        # Käytetään oletusmetodia ('ywm' tai 'ols')
        plot_pacf(y_train_stationary.dropna(), lags=n_lags, ax=axes[1], title=f'Osittaisautokorrelaatiofunktio (PACF) (d={d})', method='ywm')
        axes[1].grid(True, linestyle=':')
    except Exception as e_pacf:
        print(f"VIRHE PACF-kuvaajan piirrossa: {e_pacf}")
        axes[1].set_title('PACF-kuvaajan piirto epäonnistui')
        traceback.print_exc()


    plt.tight_layout()
    plt.show()

    print("\n--- Kuvaajien tulkintaohjeet (p ja q) ---")
    print("Tarkastele yllä olevia kuvaajia:")
    print("1. PACF-kuvaaja (alempi): Etsi kohta (lag), jonka jälkeen siniset pylväät pysyvät merkittävyysalueen (sininen/harmaa alue) sisällä ('leikkautuu').")
    print("   Tämä kohta viittaa mahdolliseen **p**-arvoon (AR-kertaluku). Jos pylväät 'häipyvät' hitaasti kohti nollaa, AR-osa on merkittävä.")
    print("2. ACF-kuvaaja (ylempi): Etsi kohta (lag), jonka jälkeen pylväät pysyvät merkittävyysalueen sisällä ('leikkautuu').")
    print("   Tämä kohta viittaa mahdolliseen **q**-arvoon (MA-kertaluku). Jos pylväät 'häipyvät' hitaasti kohti nollaa, MA-osa on merkittävä.")
    print("3. Jos molemmat näyttävät häipyvän hitaasti, kokeile pieniä p:n ja q:n arvoja (esim. p=1, q=1). Usein ARMA(1,1) on hyvä lähtökohta.")
    print("4. Huomioi mahdolliset vahvat piikit kausiluonteisilla viiveillä (esim. 24, 48, 72). Ne vahvistavat kausivaihtelun merkitystä (käsitellään seuraavissa vaiheissa).")
    print("\nPäättele kuvaajien perusteella sopivat p ja q -arvot. Voit päivittää ne `ORDER`-muuttujaan (esim. Osa 1:n solussa) ennen mallin sovitusta (Osa 7).")
    # Muistutus nykyisestä placeholder-arvosta (päivitetty d=0)
    print(f"Nykyinen ORDER placeholder (ennen p/q päivitystä): {ORDER}")
    print("\nOsa 4: ACF/PACF-analyysi (p, q) - OK")


else:
    print("VIRHE: Stationaarista harjoitusdataa ('y_train_stationary') ei löytynyt edellisestä solusta.")
    print("VAROITUS: Ei voida piirtää ACF/PACF-kuvaajia. p:n ja q:n määritys jää arvailun varaan.")

In [ ]:
# @title 5. Kausittaisen Differoinnin (D) Määritys (SARIMAX)

from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import traceback # Varmistetaan tuonti

print("--- Aloitetaan Kausittaisen Stationaarisuuden Tarkastelu ---")

# Alustetaan muuttujat
D = None # Kausittaisen differoinnin kertaluku
# Käytetään edellisessä vaiheessa määriteltyä y_train_stationary -sarjaa
y_train_final_stationary = None # Tähän tallennetaan sarja P,Q analyysia varten

# Varmistetaan edellisen vaiheen tulokset ja SEASONAL_ORDER
if 'y_train_stationary' in locals() and isinstance(y_train_stationary, pd.Series) and not y_train_stationary.empty and \
   'SEASONAL_ORDER' in locals() and isinstance(SEASONAL_ORDER, tuple) and len(SEASONAL_ORDER) == 4:

    # Haetaan kausijakson pituus (m)
    m = SEASONAL_ORDER[3]
    if not isinstance(m, int) or m <= 1:
        print(f"VIRHE: Virheellinen kausijakson pituus (m={m}) SEASONAL_ORDERissa. Oletetaan m=24.")
        m = 24
    else:
        print(f"Käytetään kausijakson pituutta m = {m}")

    # Funktio ADF-testin suorittamiseen (kopioitu/määritelty uudelleen)
    def perform_adf_test(series, series_name="Series"):
        """Suorittaa ADF-testin ja tulostaa tulokset selkeästi."""
        print(f"\nSuoritetaan ADF-testi sarjalle: {series_name}")
        try:
             # Varmista, että sarjassa on tarpeeksi arvoja testiin
             if len(series.dropna()) < int(12 * (len(series)/100)**(1/4)) + 2 : # Heuristiikka minimipituudelle
                  print(f"VAROITUS: Liian vähän dataa ({len(series.dropna())}) ADF-testiin sarjalle {series_name}. Ohitetaan testi.")
                  return None # Ei voida suorittaa testiä luotettavasti
             result = adfuller(series.dropna())
             print(f'ADF Statistic: {result[0]:.4f}')
             print(f'p-value: {result[1]:.4f}')
             print('Critical Values:')
             for key, value in result[4].items(): print(f'\t{key}: {value:.4f}')
             if result[1] <= 0.05:
                 print(f"-> Tulos: Hylätään H0 (p <= 0.05). Sarja '{series_name}' on todennäköisesti stationaarinen.")
                 return True
             else:
                 print(f"-> Tulos: Ei voida hylätä H0 (p > 0.05). Sarja '{series_name}' EI todennäköisesti ole stationaarinen.")
                 return False
        except Exception as e_adf:
             print(f"VIRHE ADF-testissä: {e_adf}")
             traceback.print_exc()
             return None # Palauta None virhetilanteessa

    # 1. Kokeile kausittaista differointia (D=1)
    # Varmista, että dataa on tarpeeksi differointiin
    if len(y_train_stationary) > m:
        print(f"\nDifferoidaan sarja kausittaisesti (lag={m})...")
        y_train_seasonal_diff1 = y_train_stationary.diff(m).dropna()

        if y_train_seasonal_diff1.empty:
            print("VIRHE: Kausittaisen differoinnin jälkeen sarja on tyhjä.")
            D = None # Estä jatko
            y_train_final_stationary = None
        else:
            # 2. Testaa kausittaisesti differoidun sarjan stationaarisuus
            is_stationary_seasonal_diff1 = perform_adf_test(y_train_seasonal_diff1, f"y_train (d={d}, D=1)")

            if is_stationary_seasonal_diff1 is None: # Jos ADF-testi epäonnistui
                 print("ADF-testi epäonnistui kausittaisesti differoidulle sarjalle. Oletetaan D=1 varovaisuussyistä.")
                 # Koska otsonilla on vahva kausivaihtelu, D=1 on todennäköisin tarve
                 D = 1
                 y_train_final_stationary = y_train_seasonal_diff1
            elif is_stationary_seasonal_diff1:
                # Jos kausittainen differointi teki sarjasta stationaarisen, käytä D=1
                D = 1
                y_train_final_stationary = y_train_seasonal_diff1
                print(f"\n-> Kausittaiseksi differoinnin kertaluvuksi (D) asetetaan: {D}")
            else:
                # Jos kausittainen differointi EI tehnyt sarjasta stationaarista
                print("\nVAROITUS: Kausittainen differointi (D=1) ei tehnyt sarjasta stationaarista ADF-testin mukaan.")
                print("Oletetaan silti D=1 otsonin tyypillisen kausivaihtelun vuoksi.")
                # Pakotetaan D=1, koska D=0 ei todennäköisesti toimi kausidatalle
                D = 1
                y_train_final_stationary = y_train_seasonal_diff1 # Käytä D=1 differoitua sarjaa
                print(f"\n-> Kausittaiseksi differoinnin kertaluvuksi (D) asetetaan: {D} (pakotettu)")
    else:
         print(f"VAROITUS: Ei tarpeeksi dataa ({len(y_train_stationary)}) kausittaiseen differointiin (lag={m}). Oletetaan D=0.")
         D = 0
         y_train_final_stationary = y_train_stationary # Käytä ei-kausidifferoitua

    # 3. Visualisoi kausittaisesti differoitu sarja (jos D=1 ja dataa on)
    if D is not None and D > 0 and y_train_final_stationary is not None and not y_train_final_stationary.empty:
        try:
            plt.figure(figsize=(14, 5))
            plt.plot(y_train_final_stationary.index, y_train_final_stationary, label=f'y_train (d={d}, D={D})', color='green')
            plt.title(f'Kausittaisesti ({m}h) Differoitu Aikasarja')
            plt.legend()
            plt.grid(True, linestyle=':')
            plt.tight_layout()
            plt.show()
        except Exception as e_vis:
             print(f"VIRHE kausidifferoidun sarjan visualisoinnissa: {e_vis}")
    elif D == 0:
        print("\nKausittaista differointia (D=0) ei tehty (tai ei tarvittu).")
    elif y_train_final_stationary is None or y_train_final_stationary.empty:
         print("\nEi voida visualisoida, koska lopullinen stationaarinen sarja puuttuu tai on tyhjä.")

    # Päivitetään SEASONAL_ORDER-muuttuja löydetyllä D:llä
    if D is not None:
         if 'SEASONAL_ORDER' in locals() and isinstance(SEASONAL_ORDER, tuple) and len(SEASONAL_ORDER) == 4:
              # Päivitetään vain D, säilytetään aiemmat P, Q, m
              SEASONAL_ORDER = (SEASONAL_ORDER[0], D, SEASONAL_ORDER[2], m)
              print(f"\nPäivitetty SEASONAL_ORDER: {SEASONAL_ORDER} (P ja Q ovat vielä placeholdereita)")
         else:
              print(f"VAROITUS: SEASONAL_ORDER-muuttujaa ei löytynyt tai se on vääränlainen. Asetetaan SEASONAL_ORDER=(1, {D}, 1, {m}).")
              SEASONAL_ORDER = (1, D, 1, m) # Käytä oletus P=1, Q=1
    else:
         print("VAROITUS: Kausittaisen differoinnin astetta D ei voitu määrittää.")


    # Varmistetaan vielä, että y_train_final_stationary on määritelty
    if 'y_train_final_stationary' not in locals() or y_train_final_stationary is None:
         print("VIRHE: y_train_final_stationary ei ole määritelty tämän solun lopussa.")
         # Yritetään palauttaa y_train_stationary, jos D=0
         if D==0 and 'y_train_stationary' in locals():
              y_train_final_stationary = y_train_stationary
              print("Asetettiin y_train_final_stationary takaisin y_train_stationaaryksi (koska D=0).")
         else:
              print("Ei voida jatkaa ilman stationaarista dataa.")


    print("\nOsa 5: Kausittaisen Differoinnin (D) Määritys - OK")

else:
    print("VIRHE: Stationaarista dataa ('y_train_stationary') tai SEASONAL_ORDER ei löytynyt edellisistä soluista.")
    # Asetetaan D ja data Noneksi, jos data puuttuu
    D = None
    y_train_final_stationary = None

In [ ]:
# @title 6. Kausittaisten Kertalukujen (P, Q) Määritys (ACF/PACF) (SARIMAX)

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
import traceback
import pandas as pd # Varmistetaan pandas tuonti
import numpy as np # Varmistetaan numpy tuonti

print("--- Määritetään Kausittaisia ARIMA kertalukuja P ja Q ---")

# Varmistetaan, että lopullinen stationaarinen data ja parametrit ovat olemassa
if 'y_train_final_stationary' in locals() and isinstance(y_train_final_stationary, pd.Series) and not y_train_final_stationary.empty and \
   'SEASONAL_ORDER' in locals() and isinstance(SEASONAL_ORDER, tuple) and len(SEASONAL_ORDER) == 4 and \
   'd' in locals() and d is not None and 'D' in locals() and D is not None: # Lisätty d ja D tarkistus

    m = SEASONAL_ORDER[3] # Kausijakson pituus

    # Asetetaan piirrettävien viiveiden (lags) määrä
    # Tarpeeksi pitkä nähdäkseen useita kausijaksoja (esim. 3*m)
    # Varmistetaan, ettei lagien määrä ylitä puolta datan pituudesta
    n_lags = min(72, len(y_train_final_stationary) // 2 - 1)
    if n_lags < m: # Varmista että näkyy vähintään yksi kausijakso
         # Jos dataa on hyvin vähän, käytä pienempää lagia
         n_lags = min(m, len(y_train_final_stationary) // 2 - 1)
         if n_lags <= 0: # Jos ei voida määrittää järkevää lagia
              print("VAROITUS: Liian vähän dataa ACF/PACF-kuvaajien piirtoon.")
              n_lags = min(10, len(y_train_final_stationary)-1) # Yritä edes jotain pientä

    if n_lags > 0:
        print(f"\nPiirretään ACF ja PACF {n_lags} viiveellä sarjalle (d={d}, D={D})...")
        print(f"Tarkastellaan erityisesti viiveitä {m}, {2*m}, {3*m}...")

        fig, axes = plt.subplots(2, 1, figsize=(12, 8))

        # Autokorrelaatiofunktio (ACF) - auttaa määrittämään Q:n kausittaisilla viiveillä
        try:
            plot_acf(y_train_final_stationary.dropna(), lags=n_lags, ax=axes[0], title=f'ACF sarjalle (d={d}, D={D})')
            axes[0].grid(True, linestyle=':')
            # Korostetaan kausiviiveitä
            for k in range(1, n_lags // m + 1):
                if k*m <= n_lags: # Varmista ettei ylitetä max lagia
                     axes[0].axvline(x=k*m, color='r', linestyle='--', alpha=0.7, label=f'Lag {k*m}' if k == 1 else None)
            if m <= n_lags : axes[0].legend(handles=axes[0].get_lines()[-1:], loc='upper right') # Näytä legenda vain kerran

        except Exception as e_acf:
            print(f"VIRHE ACF-kuvaajan piirrossa: {e_acf}")
            axes[0].set_title('ACF-kuvaajan piirto epäonnistui')
            traceback.print_exc()


        # Osittaisautokorrelaatiofunktio (PACF) - auttaa määrittämään P:n kausittaisilla viiveillä
        try:
            plot_pacf(y_train_final_stationary.dropna(), lags=n_lags, ax=axes[1], title=f'PACF sarjalle (d={d}, D={D})', method='ywm')
            axes[1].grid(True, linestyle=':')
            # Korostetaan kausiviiveitä
            for k in range(1, n_lags // m + 1):
                 if k*m <= n_lags:
                      axes[1].axvline(x=k*m, color='r', linestyle='--', alpha=0.7, label=f'Lag {k*m}' if k == 1 else None)
            if m <= n_lags : axes[1].legend(handles=axes[1].get_lines()[-1:], loc='upper right')

        except Exception as e_pacf:
            print(f"VIRHE PACF-kuvaajan piirrossa: {e_pacf}")
            axes[1].set_title('PACF-kuvaajan piirto epäonnistui')
            traceback.print_exc()


        plt.tight_layout()
        plt.show()

        print("\n--- Kuvaajien tulkintaohjeet (P ja Q) ---")
        print("Tarkastele yllä olevia kuvaajia KIINNITTÄEN HUOMIOTA KAUSILUONTEISIIN VIIVEISIIN (punaiset katkoviivat: 24, 48, 72):")
        print(f"1. PACF-kuvaaja (alempi): Jos pylväät 'leikkautuvat' merkittävyysalueen sisään nopeasti **kausittaisen viiveen P*m jälkeen** (esim. viiveen {m} jälkeen, mutta {2*m}:n kohdalla ovat pieniä), se viittaa **P**-arvoon.")
        print(f"2. ACF-kuvaaja (ylempi): Jos pylväät 'leikkautuvat' merkittävyysalueen sisään nopeasti **kausittaisen viiveen Q*m jälkeen** (esim. viiveen {m} jälkeen), se viittaa **Q**-arvoon.")
        print("3. Jos molemmilla kausittaisilla viiveillä pylväät 'häipyvät' hitaasti, kokeile pieniä P:n ja Q:n arvoja (esim. P=1, Q=1).")
        print("\nPäättele kuvaajien perusteella sopivat P ja Q -arvot ja päivitä ne `SEASONAL_ORDER`-muuttujaan seuraavaa vaihetta varten.")
        print(f"Nykyinen SEASONAL_ORDER placeholder (päivitetty D={D}): {SEASONAL_ORDER}")
        print("\nOsa 6: Kausittaisten ACF/PACF-analyysi (P, Q) - OK")

    else:
         print("VAROITUS: Ei voitu piirtää ACF/PACF-kuvaajia (liian vähän viiveitä tai dataa).")


else:
    print("VIRHE: Lopullista stationaarista dataa ('y_train_final_stationary') tai SEASONAL_ORDER ei löytynyt edellisistä soluista.")
    print("VAROITUS: Ei voida piirtää kausittaisia ACF/PACF-kuvaajia. P:n ja Q:n määritys jää arvailun varaan.")

In [ ]:
# @title 7. SARIMAX-Mallin Sovitus

from statsmodels.tsa.statespace.sarimax import SARIMAX
import traceback

print("--- Aloitetaan SARIMAX-mallin sovitus ---")

# Varmistetaan tarvittavien muuttujien olemassaolo
model_fit_sarimax = None # Alustetaan Noneksi

if 'y_train' in locals() and y_train is not None and not y_train.empty and \
   'X_train' in locals() and X_train is not None and not X_train.empty and \
   'ORDER' in locals() and isinstance(ORDER, tuple) and len(ORDER) == 3 and \
   'SEASONAL_ORDER' in locals() and isinstance(SEASONAL_ORDER, tuple) and len(SEASONAL_ORDER) == 4:

    # Varmistetaan, että y_train ja X_train ovat samanmittaisia
    if len(y_train) != len(X_train):
        print(f"VIRHE: y_train ({len(y_train)}) ja X_train ({len(X_train)}) pituudet eivät täsmää!")
        # Yritetään kohdistaa indeksit varmuuden vuoksi
        common_index_train = y_train.index.intersection(X_train.index)
        y_train = y_train.loc[common_index_train]
        X_train = X_train.loc[common_index_train]
        print(f"Indeksit kohdistettu, uusi pituus: {len(y_train)}")
        if len(y_train) != len(X_train) or y_train.empty:
             print("VIRHE: Kohdistus epäonnistui tai data tyhjeni. Ei voida jatkaa.")
             model_fit_sarimax = None


    # Jatka vain jos datat ovat kunnossa kohdistuksen jälkeen
    if y_train is not None and X_train is not None and len(y_train) == len(X_train) and not y_train.empty:
        print(f"Käytetään SARIMAX-järjestystä: order={ORDER}, seasonal_order={SEASONAL_ORDER}")
        print(f"Harjoitusdata: y_train={y_train.shape}, X_train={X_train.shape}")

        try:
            # 1. Alusta SARIMAX-malli
            # Lisätään exog=X_train
            # enforce_stationarity/invertibility=False voi auttaa konvergenssissa,
            # erityisesti jos parametrit ovat lähellä yksikköympyrää.
            model = SARIMAX(y_train,
                            exog=X_train,
                            order=ORDER,
                            seasonal_order=SEASONAL_ORDER,
                            enforce_stationarity=False,
                            enforce_invertibility=False)

            # 2. Sovita malli
            print("Sovitus käynnissä (voi kestää hetken)...")
            # disp=False piilottaa konvergenssitulosteet
            model_fit_sarimax = model.fit(disp=False)
            print("Sovitus valmis.")

            # 3. Tulosta mallin yhteenveto
            print("\n--- Mallin Yhteenveto (SARIMAX) ---")
            print(model_fit_sarimax.summary())
            print("----------------------------------")
            print("\nTulkintaohjeita Yhteenvetoon:")
            print("- Yhteenvedossa näkyy nyt myös exog-muuttujien kertoimet.")
            print("- Tarkastele kaikkien kertoimien merkitsevyyttä (P>|z|).")
            print("- Diagnostiikkatestit (Ljung-Box, Jarque-Bera, Heteroskedasticity) kertovat residuaalien ominaisuuksista.")
            print("- AIC/BIC auttavat mallivertailussa (pienempi on parempi).")

            print("\nOsa 7: SARIMAX-mallin sovitus - OK")

        except MemoryError as me:
             print(f"\nVIRHE: Muisti loppui kesken SARIMAX-mallin sovituksessa: {me}")
             print("Kokeile pienempiä kertalukuja (p,q,P,Q) tai vähemmän dataa.")
             model_fit_sarimax = None
        except Exception as e:
            print(f"\nOdottamaton VIRHE SARIMAX-mallin sovituksessa: {e}")
            traceback.print_exc()
            model_fit_sarimax = None

else:
    # Kerrotaan tarkemmin mikä puuttuu
    missing_vars_fit = []
    if 'y_train' not in locals() or y_train is None or y_train.empty: missing_vars_fit.append('y_train')
    if 'X_train' not in locals() or X_train is None or X_train.empty: missing_vars_fit.append('X_train')
    if 'ORDER' not in locals() or not isinstance(ORDER, tuple) or len(ORDER)!=3: missing_vars_fit.append('ORDER')
    if 'SEASONAL_ORDER' not in locals() or not isinstance(SEASONAL_ORDER, tuple) or len(SEASONAL_ORDER)!=4: missing_vars_fit.append('SEASONAL_ORDER')
    print(f"VIRHE: Mallin sovitukseen tarvittavia muuttujia puuttuu tai ne ovat tyhjiä: {missing_vars_fit}")
    model_fit_sarimax = None

In [ ]:
# @title 8. Ennusteiden Tekeminen (SARIMAX) - VAIHDETTU .predict()-metodiin

import pandas as pd
import numpy as np
import traceback

print("--- Aloitetaan ennusteiden tekeminen sovitetulla SARIMAX-mallilla (käyttäen .predict()) ---")

# Varmistetaan, että sovitettu malli, testidata (y ja X) ja train-data (pituuteen) ovat olemassa
forecast_values_sarimax = None

# Tarkistetaan tarvittavien muuttujien olemassaolo ja tyyppi
model_exists = 'model_fit_sarimax' in locals() and model_fit_sarimax is not None
y_test_exists = 'y_test' in locals() and isinstance(y_test, pd.Series) and not y_test.empty
X_test_exists = 'X_test' in locals() and isinstance(X_test, pd.DataFrame) and not X_test.empty
# Tarvitaan y_train pituus ennusteen aloituskohdan määrittämiseen .predict():lle
y_train_exists = 'y_train' in locals() and isinstance(y_train, pd.Series) and not y_train.empty

if model_exists and y_test_exists and X_test_exists and y_train_exists:

    # Varmista y_test ja X_test pituudet ja indeksit
    if len(y_test) != len(X_test):
         print(f"VAROITUS: y_test ({len(y_test)}) ja X_test ({len(X_test)}) pituudet eivät täsmää! Yritetään kohdistaa...")
         try:
              common_test_index = y_test.index.intersection(X_test.index)
              if len(common_test_index) < max(len(y_test), len(X_test)): print("  -> Indeksien leikkaus pienensi dataa.")
              y_test = y_test.loc[common_test_index]
              X_test = X_test.loc[common_test_index]
              print(f"  -> Indeksit kohdistettu testidatalle, uusi pituus: {len(y_test)}")
              if len(y_test) == 0: raise ValueError("Testidata tyhjeni indeksien kohdistuksessa.")
         except Exception as e_align:
              print(f"VIRHE: Testidatan indeksien kohdistus epäonnistui: {e_align}")
              y_test = None; X_test = None # Estä jatko

    # Jatka vain jos datat ovat kunnossa
    if y_test is not None and X_test is not None:
        try:
            # 1. Määritä ennusteen alku- ja loppuindeksit/askeleet .predict() varten
            n_train = len(y_train) # Tarvitaan y_train pituus
            n_forecast_steps = len(y_test)
            # .predict() käyttää indeksejä harjoitusdatan lopusta alkaen
            # 0..n_train-1 ovat harjoitusdataa, joten ennuste alkaa indeksistä n_train
            forecast_start_index = n_train
            forecast_end_index = n_train + n_forecast_steps - 1

            print(f"Tehdään {n_forecast_steps} askeleen ennuste testijaksolle käyttäen .predict(start={forecast_start_index}, end={forecast_end_index})...")

            # 2. Hae ulkoiset muuttujat ennustejaksolle (eli X_test)
            expected_exog_cols = model_fit_sarimax.model.exog_names
            if list(X_test.columns) != expected_exog_cols:
                 print("VAROITUS: X_test sarakkeet eivät täsmää täysin mallin odottamiin. Yritetään järjestää...")
                 try:
                      exog_forecast = X_test[expected_exog_cols].copy() # Käytä kopiota
                      print("X_test sarakkeet järjestetty.")
                 except KeyError as ke:
                      raise ValueError(f"Puuttuvia exog-sarakkeita ennustejaksolle: {ke}")
            else:
                 # Varmistetaan että otetaan oikea määrä rivejä ja käytetään kopiota
                 exog_forecast = X_test.iloc[:n_forecast_steps].copy()

            # Tarkistetaan exog_forecast muoto ja tyhjät arvot
            if exog_forecast.shape != (n_forecast_steps, len(expected_exog_cols)):
                 raise ValueError(f"exog_forecast muoto ({exog_forecast.shape}) ei vastaa odotettua ({(n_forecast_steps, len(expected_exog_cols))})")
            if exog_forecast.isnull().any().any():
                print("VAROITUS: exog_forecast sisältää NaN-arvoja! Yritetään täyttää ffill/bfill...")
                exog_forecast.ffill(inplace=True).bfill(inplace=True)
                if exog_forecast.isnull().any().any():
                     raise ValueError("Ei voitu täyttää NaN-arvoja exog_forecast datasta.")


            # 3. *** MUUTOS TÄSSÄ: Käytä .predict() ***
            print(f"Käytetään exog-dataa muodossa: {exog_forecast.shape}")
            # HUOM: .predict() vaatii exog-datan myös ennustejaksolle
            forecast_mean = model_fit_sarimax.predict(start=forecast_start_index, end=forecast_end_index, exog=exog_forecast)

            # 4. Tarkista palautettu tyyppi ja aseta indeksi
            if isinstance(forecast_mean, pd.Series):
                 forecast_values_sarimax = forecast_mean
                 # Yritetään asettaa testidatan indeksi
                 if len(forecast_values_sarimax) == len(y_test.index):
                      forecast_values_sarimax.index = y_test.index
                      print(".predict()-ennusteet laskettu ja indeksi asetettu.")
                 else:
                      print(f"VAROITUS: Ennusteiden ({len(forecast_values_sarimax)}) ja testidatan ({len(y_test)}) pituudet eivät täsmää .predict():n jälkeen.")
                      # Yritetään silti jatkaa, jos mahdollista
                      if len(forecast_values_sarimax) == len(y_test):
                           try:
                                forecast_values_sarimax.index = y_test.index
                                print("Indeksi asetettu manuaalisesti pituusmatchin perusteella.")
                           except Exception as e_idx_manual:
                                print(f"Indeksin manuaalinen asetus epäonnistui: {e_idx_manual}")
                                forecast_values_sarimax = None # Ei voida korjata
                      else: forecast_values_sarimax = None

            else:
                 print(f"VIRHE: .predict() ei palauttanut odotettua Pandas Series -objektia (tyyppi: {type(forecast_mean)})")
                 forecast_values_sarimax = None

            # 5. Tulosta ja tarkista NaN (jos ennuste onnistui)
            if forecast_values_sarimax is not None:
                print("\nEnnusteiden alku (SARIMAX - predict):")
                print(forecast_values_sarimax.head())
                print("\nEnnusteiden loppu (SARIMAX - predict):")
                print(forecast_values_sarimax.tail())

                nan_in_forecast = forecast_values_sarimax.isnull().sum()
                if nan_in_forecast > 0:
                    print(f"\nVAROITUS: SARIMAX .predict()-ennuste sisältää {nan_in_forecast} NaN-arvoa!")
                else:
                    print("\nEnnusteet eivät sisällä NaN-arvoja.")

        except Exception as e:
            print(f"\nOdottamaton VIRHE ennusteiden tekemisessä (.predict()): {e}")
            traceback.print_exc()
            forecast_values_sarimax = None
    # else: (implisiittinen, jos y_test tai X_test oli None kohdistuksen jälkeen)
    #    print("Ennustamista ei voida jatkaa puuttuvan testidatan vuoksi.")

else:
    missing_vars_fc = []
    if 'model_fit_sarimax' not in locals() or model_fit_sarimax is None: missing_vars_fc.append('model_fit_sarimax')
    if 'y_test' not in locals() or not isinstance(y_test, pd.Series) or y_test.empty: missing_vars_fc.append('y_test')
    if 'X_test' not in locals() or not isinstance(X_test, pd.DataFrame) or X_test.empty: missing_vars_fc.append('X_test')
    if 'y_train' not in locals() or not isinstance(y_train, pd.Series) or y_train.empty: missing_vars_fc.append('y_train (needed for start index)')
    print(f"VIRHE: Ennusteiden tekemiseen tarvittavia muuttujia puuttuu tai ne ovat tyhjiä: {missing_vars_fc}.")
    forecast_values_sarimax = None


print("\nOsa 8: Ennusteiden tekeminen (SARIMAX - predict) - OK (tai epäonnistunut)")

In [ ]:
# @title 9. Tulosten Arviointi ja Vertailu (SARIMAX)

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import traceback

# --- VARMISTETAAN PARAMETRIEN MÄÄRITTELY TÄSSÄ SOLUSSA ---
# Asetetaan kynnysarvo (voit muuttaa tätä tarvittaessa, esim. 85)
O3_THRESHOLD_8H_AVG = 85 # µg/m³ (Vaikka virallinen varoitusraja)
if 'PREDICTION_HORIZON' not in locals(): PREDICTION_HORIZON = 24
# Haetaan kertaluvut tulostusta varten
display_order = ORDER if 'ORDER' in locals() else ('?','?','?')
display_seasonal_order = SEASONAL_ORDER if 'SEASONAL_ORDER' in locals() else ('?','?','?', '?')
# ----------------------------------------------------------

print("--- Aloitetaan SARIMAX-ennusteiden arviointi ---")

# Alustetaan muuttujat
evaluation_possible = False
rmse_sarimax = None; mae_sarimax = None
rmse_baseline = None; mae_baseline = None
baseline_forecast_sarimax = None

# Tarkistetaan ennusteiden ja testidatan olemassaolo ja tyyppi
# Käytetään forecast_values_sarimax -muuttujaa edellisestä solusta
if 'forecast_values_sarimax' in locals() and isinstance(forecast_values_sarimax, pd.Series) and not forecast_values_sarimax.empty and \
   'y_test' in locals() and isinstance(y_test, pd.Series) and not y_test.empty:

    # Varmistetaan sama pituus
    if len(forecast_values_sarimax) == len(y_test):
        print(f"Arvioidaan {len(y_test)} ennustepistettä.")
        evaluation_possible = True
        common_index = y_test.index
        try:
            # Kohdista indeksit ja tarkista/täytä NaN
            forecast_eval = forecast_values_sarimax.reindex(common_index)
            test_eval = y_test.reindex(common_index) # Käytetään y_test:iä

            if forecast_eval.isnull().any():
                 print("VAROITUS: NaN-arvoja SARIMAX-ennusteessa indeksin kohdistuksen jälkeen. Täytetään ffill/bfill.")
                 forecast_eval = forecast_eval.ffill().bfill()
            if test_eval.isnull().any():
                 print("VAROITUS: NaN-arvoja testidatassa indeksin kohdistuksen jälkeen. Täytetään ffill/bfill.")
                 test_eval = test_eval.ffill().bfill()

            if forecast_eval.isnull().any() or test_eval.isnull().any():
                 print("VIRHE: Ei voitu täyttää kaikkia NaN-arvoja. Arviointi epäonnistuu.")
                 evaluation_possible = False
            elif len(forecast_eval) != len(test_eval):
                 print("VIRHE: Pituudet eivät täsmää kohdistuksen/täytön jälkeen.")
                 evaluation_possible = False
            else:
                 print("Indeksit kohdistettu ja NaN-arvot tarkistettu/täytetty.")

        except Exception as e_reindex_eval:
            print(f"VIRHE indeksin kohdistuksessa tai NaN-täytössä arvioinnissa: {e_reindex_eval}")
            evaluation_possible = False
    else:
        print(f"VIRHE: SARIMAX-ennusteiden ({len(forecast_values_sarimax)}) ja testidatan ({len(y_test)}) pituudet eivät täsmää.")

else:
    missing_eval_vars = []
    if 'forecast_values_sarimax' not in locals() or not isinstance(forecast_values_sarimax, pd.Series) or forecast_values_sarimax.empty:
         missing_eval_vars.append('forecast_values_sarimax')
    if 'y_test' not in locals() or not isinstance(y_test, pd.Series) or y_test.empty:
         missing_eval_vars.append('y_test')
    print(f"VIRHE: Arviointiin tarvittavia muuttujia puuttuu tai ne ovat tyhjiä: {missing_eval_vars}.")

# --- Metriikoiden laskenta (vain jos evaluation_possible on True) ---
if evaluation_possible:
    try:
        # --- Baseline-laskenta ---
        def calculate_baseline_persistence_sarimax(targets_series):
            """Laskee naiivin persistenssi-baselinen Seriesille."""
            print("Lasketaan Baseline-ennuste (jakson eka arvo toistuu)...")
            try:
                if targets_series.empty: return pd.Series(dtype=float)
                first_val = targets_series.iloc[0]
                baseline_preds_np = np.repeat(first_val, len(targets_series))
                baseline_preds = pd.Series(baseline_preds_np, index=targets_series.index)
                return baseline_preds
            except Exception as e_base:
                print(f"VIRHE baseline-laskennassa: {e_base}")
                return None

        # Käytä test_evalia baselineen (varmistettu ettei NaN)
        baseline_forecast_sarimax = calculate_baseline_persistence_sarimax(test_eval)

        # --- Regressiometriikat ---
        print("\nLasketaan regressiometriikat...")
        rmse_sarimax = np.sqrt(mean_squared_error(test_eval, forecast_eval))
        mae_sarimax = mean_absolute_error(test_eval, forecast_eval)
        print(f"\n--- SARIMAX{display_order}x{display_seasonal_order} Mallin Arviointi ---")
        print(f"RMSE: {rmse_sarimax:.4f} µg/m³")
        print(f"MAE:  {mae_sarimax:.4f} µg/m³")

        if baseline_forecast_sarimax is not None and len(baseline_forecast_sarimax) == len(test_eval):
            if not baseline_forecast_sarimax.isnull().any(): # Tarkistus baseline NaN:lle
                rmse_baseline = np.sqrt(mean_squared_error(test_eval, baseline_forecast_sarimax))
                mae_baseline = mean_absolute_error(test_eval, baseline_forecast_sarimax)
                print(f"\n--- Baseline-Mallin Arviointi (Naiivi Persistenssi) ---")
                print(f"(Käytetään samaa baselinea vertailuun)")
                print(f"RMSE: {rmse_baseline:.4f} µg/m³")
                print(f"MAE:  {mae_baseline:.4f} µg/m³")

                # --- Vertailu ---
                print("\n--- Vertailu Baselineen ---")
                if rmse_sarimax is not None and rmse_baseline is not None:
                    improvement_rmse = rmse_baseline - rmse_sarimax
                    print(f"SARIMAX vs Baseline RMSE: {improvement_rmse:+.4f} µg/m³ ({'SARIMAX parempi' if improvement_rmse > 0 else 'Baseline parempi tai sama'})")
                else: print("RMSE-vertailua ei voida tehdä.")
                if mae_sarimax is not None and mae_baseline is not None:
                    improvement_mae = mae_baseline - mae_sarimax
                    print(f"SARIMAX vs Baseline MAE:  {improvement_mae:+.4f} µg/m³ ({'SARIMAX parempi' if improvement_mae > 0 else 'Baseline parempi tai sama'})")
                else: print("MAE-vertailua ei voida tehdä.")
            else: print("\nBaseline-metriikoita ei voitu laskea NaN-arvojen vuoksi.")
        else: print("\nBaseline-ennustetta ei voitu laskea tai sen pituus ei täsmää.")

        # --- 8h Liukuvan keskiarvon arviointi ---
        print(f"\nLasketaan 8h liukuvia keskiarvoja koko testijaksolle ja verrataan kynnysarvoon ({O3_THRESHOLD_8H_AVG} µg/m³)...")
        try:
            actual_series_8h = test_eval.rolling(window=8, min_periods=1).mean()
            pred_series_8h = forecast_eval.rolling(window=8, min_periods=1).mean()
            actual_warning_hours = actual_series_8h > O3_THRESHOLD_8H_AVG
            pred_warning_hours = pred_series_8h > O3_THRESHOLD_8H_AVG
            n_samples_eval = len(test_eval)

            print("\n--- 8h Liukuvan Keskiarvon Varoitustason Ylityksen Arviointi (SARIMAX - Tuntitaso) ---")
            print(f"Todellisia varoitustunteja testidatassa (> {O3_THRESHOLD_8H_AVG} µg/m³): {actual_warning_hours.sum()} / {n_samples_eval}")
            print(f"Ennustettuja varoitustunteja (SARIMAX):                   {pred_warning_hours.sum()} / {n_samples_eval}")

            if actual_warning_hours.sum() > 0 or pred_warning_hours.sum() > 0:
                print("\nSekaannusmatriisi (Confusion Matrix) varoituksille (SARIMAX - Tuntitaso):")
                cm = confusion_matrix(actual_warning_hours, pred_warning_hours, labels=[False, True])
                cm_df = pd.DataFrame(cm, index=['Todellinen EI Varoitusta', 'Todellinen KYLLÄ Varoitus'],
                                   columns=['Ennuste EI', 'Ennuste KYLLÄ'])
                print(cm_df)
                print("\nLuokitteluraportti varoituksille (SARIMAX - Tuntitaso):")
                report = classification_report(actual_warning_hours, pred_warning_hours, target_names=['Ei Varoitusta', 'Varoitus'], labels=[False, True], zero_division=0)
                print(report)
                TN, FP, FN, TP = cm.ravel()
                recall_warning = TP / (TP + FN) if (TP + FN) > 0 else 0
                precision_warning = TP / (TP + FP) if (TP + FP) > 0 else 0
                print(f"\n---> TÄRKEIMMÄT VAROITUSMETRIIKAT (Tuntitaso):")
                print(f"  Recall (Herkkyys) 'Varoitus'-luokalle: {recall_warning:.4f}")
                print(f"  Precision (Tarkkuus) 'Varoitus'-luokalle: {precision_warning:.4f}")
            else:
                print(f"\nEi todellisia eikä ennustettuja varoitustunteja testidatassa kynnysarvolla {O3_THRESHOLD_8H_AVG} µg/m³.")

            print("\nOsa 9: Tulosten Arviointi (SARIMAX) - OK")

        except Exception as e_8h:
            print(f"VIRHE 8h liukuvan keskiarvon laskennassa tai arvioinnissa: {e_8h}")
            traceback.print_exc()

    except Exception as e:
        print(f"\nVIRHE tulosten arvioinnissa: {e}")
        traceback.print_exc()
else:
     print("\nArviointia ei voida suorittaa, koska SARIMAX-ennusteet tai testidata puuttuvat/ovat virheellisiä.")